<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [2]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2020-05-17 15:47:13--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip.2’

100%[======================================>] 261,483,817 39.7MB/s   in 6.3s   

2020-05-17 15:47:19 (39.5 MB/s) - ‘concrete_data_week4.zip.2’ saved [261483817/261483817]



In [7]:
# !unzip concrete_data_week4.zip

Archive:  concrete_data_week4.zip
replace concrete_data_week4/valid/positive/16679_1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

## Load the necessary libraries and packages

In [8]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [10]:
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

In [11]:
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

## Set the GLOBAL variables

In [12]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100


## Build and Compile

In [13]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [14]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [15]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')


Found 9501 images belonging to 2 classes.


In [16]:
model = Sequential()

In [17]:
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

Instructions for updating:
Colocations handled automatically by placer.


In [18]:
model.add(Dense(num_classes, activation='softmax'))

In [19]:
model.layers

In [20]:
model.layers[0].layers

In [21]:
model.layers[0].trainable = False

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [23]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Fit the Model



In [24]:
num_epochs = 2
steps_per_epoch_training = 10
steps_per_epoch_validation = 5

In [25]:
VGG16_model = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
10/10 [==============================] - 534s 53s/step - loss: 0.7609 - acc: 0.5270 - val_loss: 0.5658 - val_acc: 0.6560
Epoch 2/2
10/10 [==============================] - 533s 53s/step - loss: 0.4334 - acc: 0.7960 - val_loss: 0.2891 - val_acc: 0.9180


## Save the model for later use

In [26]:
model.save('classifier_vgg16_model.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [27]:
from keras.models import load_model

model_RN50 = load_model('classifier_resnet_model.h5')
# summarize model.
model_RN50.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 23,538,690
Non-trainable params: 53,120
_________________________________________________________________


/opt/conda/envs/Python36/lib/python3.6/site-packages/keras/engine/saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [28]:
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    batch_size=20,
    shuffle = False)

Found 500 images belonging to 2 classes.


## ResNet50 test evaluation

In [29]:
score = model.evaluate_generator(test_generator, verbose=1, steps=5)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

5/5 [==============================] - 57s 11s/step
acc: 84.00%


## VGG16 test evaluation

In [30]:
model_VGG16 = load_model('classifier_vgg16_model.h5')
# summarize model.
model_VGG16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 14,715,714
Non-trainable params: 0
_________________________________________________________________


In [31]:
score = model.evaluate_generator(test_generator, verbose=1, steps=5)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

5/5 [==============================] - 58s 12s/step
acc: 92.00%


## Discussion

ResNet50 reported an 84% accuracy
VGG16 reported a 92% accuracy



<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [32]:
predict = model.predict_generator(test_generator,5)

## Look at the predict data and run report as requested

In [33]:
import pandas as pd
dfpredict = pd.DataFrame(predict)
dfpredict.head()

,0,1
0,0.796342,0.203657
1,0.910927,0.089073
2,0.820921,0.179079
3,0.407363,0.592637
4,0.644152,0.355848


In [34]:
x = 0
for x in range (5):
    if predict[x,0] > .50: # if negative prediction greater than .5 declare as negative image
        print("Negative")
    else:
        print("Positive")
       
              

Negative
Negative
Negative
Positive
Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).